In [13]:
import numpy as np
import random
import json
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

In [17]:
from nltk_functions import bag_of_words, tokenize, stem
from model import NeuralNet

In [21]:
with open('data.json', 'r') as f:
    intents = json.load(f)

In [23]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/sarathdevsahadevan/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [24]:
all_words = []
tags = []
xy = []
# loop through each sentence in our intents patterns
for intent in intents['intents']:
    tag = intent['tag']
    # add to tag list
    tags.append(tag)
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = tokenize(pattern)
        # add to our words list
        all_words.extend(w)
        # add to xy pair
        xy.append((w, tag))

In [25]:
# stem and lower each word
ignore_words = ['?', '.', '!']
all_words = [stem(w) for w in all_words if w not in ignore_words]
# remove duplicates and sort
all_words = sorted(set(all_words))
tags = sorted(set(tags))

In [26]:
print(len(xy), "patterns")
print(len(tags), "tags:", tags)
print(len(all_words), "unique stemmed words:", all_words)

201 patterns
105 tags: ['2fa, security', '360 account', 'Change online banking access code', 'account closure', 'account linking', 'account overdraft', 'account statement', 'activate overseas card usage, deactivate overseas card usage', 'add payee, delete payee, bill payment', 'add payee, delete payee, fund transfer', 'add payee, delete payee, overseas fund transfer', 'airport transfer', 'annual fee, annual fee waiver', 'apply, digital banking', 'atm dynamic currency conversion', 'atm, withdraw overseas', 'autowise, auto wise', 'autowise, insure, commercial vehicle', 'bill payment', 'billing organisations', 'branch locations', 'budgeting tools', 'business accounts', 'cancel fire insurance', 'cashier order', 'charitable donations', 'check balance', 'conversion rate, forex fee, conversion fee', 'corporate cards', 'credit card application', 'credit card rewards', 'credit limit increase', 'credit score inquiry', 'critical illness', 'customer service hours', 'debit card activation', 'dynami

In [27]:
# create training data
X_train = []
y_train = []
for (pattern_sentence, tag) in xy:
    # X: bag of words for each pattern_sentence
    bag = bag_of_words(pattern_sentence, all_words)
    X_train.append(bag)
    # y: PyTorch CrossEntropyLoss needs only class labels, not one-hot
    label = tags.index(tag)
    y_train.append(label)

In [28]:
X_train = np.array(X_train)
y_train = np.array(y_train)

In [29]:
# Hyper-parameters 
num_epochs = 1000
batch_size = 8
learning_rate = 0.001
input_size = len(X_train[0])
hidden_size = 8
output_size = len(tags)
print(input_size, output_size)

396 105


In [30]:
class ChatDataset(Dataset):

    def __init__(self):
        self.n_samples = len(X_train)
        self.x_data = X_train
        self.y_data = y_train

    # support indexing such that dataset[i] can be used to get i-th sample
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    # we can call len(dataset) to return the size
    def __len__(self):
        return self.n_samples

In [31]:
dataset = ChatDataset()
train_loader = DataLoader(dataset=dataset,
                          batch_size=batch_size,
                          shuffle=True,
                          num_workers=0)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = NeuralNet(input_size, hidden_size, output_size).to(device)


In [32]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [33]:
# Train the model
for epoch in range(num_epochs):
    for (words, labels) in train_loader:
        words = words.to(device)
        labels = labels.to(dtype=torch.long).to(device)
        
        # Forward pass
        outputs = model(words)
        # if y would be one-hot, we must apply
        # labels = torch.max(labels, 1)[1]
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    if (epoch+1) % 100 == 0:
        print (f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')


print(f'final loss: {loss.item():.4f}')

Epoch [100/1000], Loss: 0.1685
Epoch [200/1000], Loss: 0.0030
Epoch [300/1000], Loss: 0.0034
Epoch [400/1000], Loss: 0.0002
Epoch [500/1000], Loss: 0.0000
Epoch [600/1000], Loss: 0.0000
Epoch [700/1000], Loss: 0.0000
Epoch [800/1000], Loss: 0.0000
Epoch [900/1000], Loss: 0.0000
Epoch [1000/1000], Loss: 0.0000
final loss: 0.0000


In [34]:
data = {
"model_state": model.state_dict(),
"input_size": input_size,
"hidden_size": hidden_size,
"output_size": output_size,
"all_words": all_words,
"tags": tags
}

In [35]:
FILE = "data.pth"
torch.save(data, FILE)

print(f'file saved to {FILE}')

file saved to data.pth
